**Introduction**
=====
Start date : 2022.08.24
> 데이터를 살펴보고 어떠한 데이터인지(사내 데이터, 사외 데이터 크롤링 DB)파악하고 회사 형태를 가정한다.
---------------------------------------

1. **필수 포함 내용**
- 지역에 따라서 선호하는 게임 장르가 다를까 라는 질문에 대답
- 연도별 게임의 트렌드가 있을까 라는 질문에 대답
- 출고량이 높은 게임에 대한 분석 및 시각화 프로세스가 포함

2. **사전 주요 가정**
* 사내 데이터인가 사외 데이터인가? -> 사외 데이터로 추정 
   - 사내 데이터라면 매출 추이 확인 가능
   - 사외 데이터라면 거시적 추세 위주로 탐색 
   
   
* 회사의 형태는? -> 괜찮은 IP 한개 보유한 중견 게임 개발사.
    * 게임 배급 및 유통사?(like 넥슨) 게임 개발사? (like CD project)
    * 대기업? 중견? 중소? (네이버, 카카오 게임즈 / 크래프톤, 블루홀 / 게임빌, 그 외 소규모)
    * 한 장르 주력? 여러 장르 한 IP로 다양하게? (Like 데브시스터즈)


In [81]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [82]:
# Load pakages

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.options.display.max_rows = 40

In [83]:
# Data acquisition

path = r'/content/drive/MyDrive/0.Data for analysis/Project/vgames2.csv'

raw = pd.read_csv(path, index_col='Unnamed: 0')
print(raw.shape)

(16598, 9)


In [84]:
raw.sample(10)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
5000,Thrillville,PS2,2006.0,Strategy,Atari,0.55,0.02,0,0.09
8319,Ys: The Ark of Napishtim,PSP,2006.0,Role-Playing,Konami Digital Entertainment,0.04,0,0,0
4796,Tim Burton's The Nightmare Before Christmas: O...,XB,2005.0,Adventure,Disney Interactive Studios,0.06,0.02,0,0
5650,J-League Pro Soccer Club o Tsukurou! 7 Euro Plus,PSP,2011.0,Sports,Sega,0,0,0.21,0
2401,SimEarth: The Living Planet,SNES,1991.0,Simulation,Imagineer,0,0,160K,0
11132,The Smurfs,DS,2011.0,Action,Ubisoft,0.24,0.14,0,0.04
3235,Amatsumi Sora ni! Kumo no Hatate ni,PSV,2015.0,Adventure,Prototype,0,0,20K,0
2360,Stubbs the Zombie in Rebel Without a Pulse,XB,2005.0,Action,THQ,0.08,0.02,0,0
1442,Major League Baseball 2K12,PS3,2012.0,Sports,Take-Two Interactive,0.18,0,0,0.01
4156,Smash Court Tennis Pro Tournament,PS2,2002.0,Sports,Sony Computer Entertainment,0.08,0.07,0,0.02


**Data Preparation**
====

### 1. Handdling Missing value
- Year   # 271
- Genre  # 50
- Publisher # 58 -> 우리의 분석에서 필요한가? 추후 필요하면 추가           

### 2. Duplicates
- name 중복되는 경우   
   
### 3. Inaccurate data

**Datatype 변경**
- Platform, Genre -> Categorical
- Year -> Int
- NA~other Sales -> float

**Outlier (이상치 조정)**
- Year 0~100 처리

In [85]:
# 분석에 사용할 데이터
df = raw.copy()

# 이름에 따라 데이터 재 정렬
df = df.sort_values(by = 'Name', ascending = True).reset_index(drop = True)

## 1. Handdling Missing value

Genre : 제거   
Year : 일단 제거   
Publisher : 분석에 안쓸 것이므로 결측값은 'Unknown'으로 대채
* 둘 다 시간 되면 외부데이터를 통해 결측치를 채워 넣는다.

In [86]:
# Missing value 확인
df.isna().sum()

Name             0
Platform         0
Year           271
Genre           50
Publisher       58
NA_Sales         0
EU_Sales         0
JP_Sales         0
Other_Sales      0
dtype: int64

In [87]:
df[df['Genre'].isna() | df['Year'].isna()]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
147,AKB1/48: Idol to Guam de Koishitara...,X360,NaN,Misc,NaN,0,0,0.01,0
198,Action Man-Operation Extreme,PS,NaN,Action,NaN,0.05,0.03,0,0.01
214,Advance Wars: Days of Ruin,DS,NaN,Strategy,Nintendo,0.44,0.13,0,0.06
217,Adventure,2600,NaN,Adventure,Atari,1.21,0.08,0,0.01
250,Agarest Senki: Re-appearance,PS3,NaN,Role-Playing,Idea Factory,0,0,0.01,0
...,...,...,...,...,...,...,...,...,...
16405,Yu Yu Hakusho: Dark Tournament,PS2,NaN,Fighting,NaN,0.1,0.08,0,0.03
16412,Yu-Gi-Oh! 5D's Wheelie Breakers (JP sales),Wii,NaN,Racing,Unknown,0,0,0.02,0
16498,Zero: Tsukihami no Kamen,Wii,NaN,Action,Nintendo,0,0,0.08,0
16568,eJay Clubworld,PS2,NaN,Misc,Empire Interactive,0.07,0.06,0,0.02


In [88]:
# 결측치 처리
df_clean = df.dropna(subset = ['Genre','Year'])

# 확인
df_clean[df_clean['Genre'].isna() | df_clean['Year'].isna()]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales


In [89]:
# Publisher 결측치 대채
# df[df['Publisher'].isna()]
df_clean = df_clean.replace(np.nan, 'Unknown')

# 확인
df_clean[df_clean['Publisher'].isna()]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales


## 2. Duplicates

In [90]:
# duplicates 확인
print(df_clean.shape[0] - df_clean['Name'].nunique()) #중복값 4947개 -> 전처리 후 다시 확인, Handle 하기.

4947


In [91]:
df_clean[df_clean.duplicated(subset = ['Name','Platform'], keep = False)]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
7712,Madden NFL 13,PS3,2012.0,Sports,Electronic Arts,2.11,0.23,0,0.22
7716,Madden NFL 13,PS3,2012.0,Sports,Electronic Arts,0,0.01,0,0
9621,Need for Speed: Most Wanted,PC,2012.0,Racing,Electronic Arts,0,0.06,0,0.02
9622,Need for Speed: Most Wanted,PC,2005.0,Racing,Electronic Arts,20K,0.23,0,0.05
9624,Need for Speed: Most Wanted,X360,2005.0,Racing,Electronic Arts,1,0.17,0.02,0.1
9630,Need for Speed: Most Wanted,X360,2012.0,Racing,Electronic Arts,0.62,0.8,0.01,0.15


**중복값을 보면 겹친 두 값 중 주로 후자에 나온 값이 데이터를 더욱 많이 보존하고 있음. 따라서 keep = 'last'로 설정**

In [92]:
# 진짜 중복값 (Name과 Platform 둘다 동일한 중복값)
df_duplicates = df_clean[df_clean.duplicated(subset = ['Name','Platform'], keep = 'last')] 
print(f'# of duplicates : {df_duplicates.shape[0]}')

# 중복값 저장
df_duplicates

# of duplicates : 3


,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
7712,Madden NFL 13,PS3,2012.0,Sports,Electronic Arts,2.11,0.23,0,0.22
9621,Need for Speed: Most Wanted,PC,2012.0,Racing,Electronic Arts,0,0.06,0,0.02
9624,Need for Speed: Most Wanted,X360,2005.0,Racing,Electronic Arts,1,0.17,0.02,0.1


In [93]:
# 중복값 처리
df_clean = df_clean.drop_duplicates(subset = ['Name','Platform'], keep = 'last')

# 최종 확인
df_clean[df_clean.duplicated(subset = ['Name','Platform'], keep = 'last')] 

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales


## 3. Inaccurate data

###**Data type transformataion**
* Year -> int
* Platform / Genre -> Category
* Sales -> float (Unit : Million)

In [94]:
# Dtype 변경 - 이상치 탐색을 위해
# Year
df_clean['Year'] = df_clean['Year'].astype('int')

# Platform / Genre
df_clean['Platform'] = df_clean['Platform'].astype('category')
df_clean['Genre'] = df_clean['Genre'].astype('category')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [95]:
# 이상한 sales 값 찾기
import re

def not_num(a):
  return bool(re.search('[A-z]', a))

error = df['NA_Sales'].apply(not_num)
df['NA_Sales'][error]

188      0.37M
233        60K
340      0.22M
483        90K
552      0.02M
         ...  
16340       0M
16383      60K
16454       0M
16464       0M
16554    0.14M
Name: NA_Sales, Length: 200, dtype: object

In [96]:
# 문자 숫자 대채 함수 (단위: M)

def cleaner(col):

  temp_list = []

  for row in col:
    if bool(re.search('K', row)) == True:
      K = float(row.replace('K','')) * 0.001
      temp_list.append(K)

    elif bool(re.search('M', row)) == True:
      M = float(row.replace('M',''))
      temp_list.append(M)

    else : temp_list.append(float(row))

  return temp_list

In [97]:
df1 = df_clean.copy()

# 함수 적용
for col in range(5,9): 
  df1.iloc[:,col] = cleaner(df1.iloc[:,col])

# 확인
df1.info()
df1

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16274 entries, 0 to 16597
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Name         16274 non-null  object  
 1   Platform     16274 non-null  category
 2   Year         16274 non-null  int64   
 3   Genre        16274 non-null  category
 4   Publisher    16274 non-null  object  
 5   NA_Sales     16274 non-null  float64 
 6   EU_Sales     16274 non-null  float64 
 7   JP_Sales     16274 non-null  float64 
 8   Other_Sales  16274 non-null  float64 
dtypes: category(2), float64(4), int64(1), object(2)
memory usage: 1.0+ MB


,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,'98 Koshien,PS,1998,Sports,Magical Company,0.15,0.10,0.12,0.03
1,.hack//G.U. Vol.1//Rebirth,PS2,2006,Role-Playing,Namco Bandai Games,0.00,0.00,0.17,0.00
2,.hack//G.U. Vol.2//Reminisce,PS2,2006,Role-Playing,Namco Bandai Games,0.11,0.09,0.00,0.03
3,.hack//G.U. Vol.2//Reminisce (jp sales),PS2,2006,Role-Playing,Namco Bandai Games,0.00,0.00,0.16,0.00
4,.hack//G.U. Vol.3//Redemption,PS2,2007,Role-Playing,Namco Bandai Games,0.00,0.00,0.17,0.00
...,...,...,...,...,...,...,...,...,...
16592,thinkSMART: Chess for Kids,DS,2011,Misc,Mentor Interactive,0.01,0.00,0.00,0.00
16593,uDraw Studio,Wii,2010,Misc,THQ,1.67,0.58,0.00,0.20
16594,uDraw Studio: Instant Artist,X360,2011,Misc,THQ,0.01,0.01,0.00,0.00
16595,uDraw Studio: Instant Artist,Wii,2011,Misc,THQ,0.08,0.09,0.00,0.02


### **Eliminating inaccurate data**
1. Year
* Year =< 0~22 은 앞에 20 이 전산오류로 빠진 것으로 추정
> 해당 값들에 2000을 더한다.
* Year > 0~22
> 해당 값들에 1900을 더한다.

In [98]:
# Year 이상치 변경 함수
def year_normal(col):
  tmp = []

  for row in col:
    if row <= 22:
      tmp.append(row + 2000)
    elif  row <= 100 :
      tmp.append(row + 1900)
    else : tmp.append(row)

  return tmp

In [99]:
# 이상치 변경
df1['Year'] = year_normal(df1['Year'])

# 확인
df1['Year'].sort_values()  # 시간 범위는 1980 ~ 2020

1526     1980
1466     1980
2891     1980
4758     1980
702      1980
         ... 
3078     2016
10306    2017
1544     2017
10307    2017
6092     2020
Name: Year, Length: 16274, dtype: int64

## 구조적 문제

In [100]:
# 컬럼이름 재정의
df_str = df1.rename(columns = {'NA_Sales':'North_America', 'EU_Sales':'EU', 'JP_Sales':'JP','Other_Sales': 'Other'})

# Melt 로 sales 데이터 병합
id = ['Name', 'Platform', 'Year', 'Genre', 'Publisher']
value = ['North_America', 'EU', 'JP', 'Other']
var = 'Region'
val_name = 'Sales'

df1 = pd.melt(df_str, id_vars = id, value_vars = value, var_name = var, value_name = val_name)

In [101]:
df1.describe()

,Year,Sales
count,65096.000000,65096.000000
mean,2006.407030,0.134567
std,5.828709,0.522471
min,1980.000000,0.000000
25%,2003.000000,0.000000
50%,2007.000000,0.010000
75%,2010.000000,0.090000
max,2020.000000,41.490000


## 힘들게 정리한 데이터 저장



In [102]:
df1.to_csv('Games_cleaned.csv', index = False)

**EDA**
======
으아악 드디어 EDA다.
여기부터는 내일....

In [103]:
df = pd.read_csv('Games_cleaned.csv')

df

,Name,Platform,Year,Genre,Publisher,Region,Sales
0,'98 Koshien,PS,1998,Sports,Magical Company,North_America,0.15
1,.hack//G.U. Vol.1//Rebirth,PS2,2006,Role-Playing,Namco Bandai Games,North_America,0.00
2,.hack//G.U. Vol.2//Reminisce,PS2,2006,Role-Playing,Namco Bandai Games,North_America,0.11
3,.hack//G.U. Vol.2//Reminisce (jp sales),PS2,2006,Role-Playing,Namco Bandai Games,North_America,0.00
4,.hack//G.U. Vol.3//Redemption,PS2,2007,Role-Playing,Namco Bandai Games,North_America,0.00
...,...,...,...,...,...,...,...
65091,thinkSMART: Chess for Kids,DS,2011,Misc,Mentor Interactive,Other,0.00
65092,uDraw Studio,Wii,2010,Misc,THQ,Other,0.20
65093,uDraw Studio: Instant Artist,X360,2011,Misc,THQ,Other,0.00
65094,uDraw Studio: Instant Artist,Wii,2011,Misc,THQ,Other,0.02
